# packages

In [9]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
import pandas as pd
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_fireworks import ChatFireworks
from dotenv import load_dotenv, find_dotenv
from pprint import pprint
from langchain_core.pydantic_v1 import BaseModel
from langchain.agents import tool
from typing import Optional, Type

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

In [2]:
_ = load_dotenv(find_dotenv())

# get the data

In [3]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/pandas-dev/pandas/main/doc/data/titanic.csv"
)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


# create custom agent

In [5]:
chat = ChatFireworks(
    model="accounts/fireworks/models/mixtral-8x7b-instruct",
)

In [7]:
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


get_word_length.invoke("abc")

3

## Defining custom tool

In [18]:
class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")

class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")

class LengthInput(BaseModel):
    query: str = Field(description="should be a string with one or many words")
    

In [11]:
class CustomSearchTool(BaseTool):
    name="custom_search"
    description="useful for when you need to answer questions about current events"
    args_schema: Type[BaseModel]=SearchInput
    
    def _run(self,query:str, run_manager:Optional[CallbackManagerForToolRun]=None)->str:
        """Use the tool"""
        return "LangChain"
    
    async def _arun(self, query:str,run_manager: Optional[AsyncCallbackManagerForToolRun]=None)->str:
        """Use the tool asynchronously"""
        raise NotImplementedError("custom_search does not support async")
    



In [14]:
class CustomCalculatorTool(BaseTool):
    name="calculator"
    description = "useful for when you need to answer questions about math"
    args_schema: Type[BaseModel]=CalculatorInput
    return_direct: bool= True

    def _run(self,a:int,b:int,run_manager:Optional[CallbackManagerForToolRun]=None)->str:
        """Use the tool"""
        return a*b
    
    async def _arun(self,a:int,b:int,run_manager:Optional[AsyncCallbackManagerForToolRun]=None)->str:
        """Use the tool asynchronously"""
        raise NotImplementedError("Calculator does not support async")

In [19]:
class CustomLengthTool(BaseTool):
    name="length"
    description = "Returns the length of a word."
    args_schema: Type[BaseModel]=LengthInput
    return_direct: bool= True

    def _run(self,word:int,run_manager:Optional[CallbackManagerForToolRun]=None)->str:
        """Use the tool"""
        return str(len(word))
    
    async def _arun(self,a:int,b:int,run_manager:Optional[AsyncCallbackManagerForToolRun]=None)->str:
        """Use the tool asynchronously"""
        raise NotImplementedError("length does not support async")

In [15]:
search = CustomSearchTool()
print(search.name)
print(search.description)
print(search.args)

custom_search
useful for when you need to answer questions about current events
{'query': {'title': 'Query', 'description': 'should be a search query', 'type': 'string'}}


In [16]:
multiply = CustomCalculatorTool()
print(multiply.name)
print(multiply.description)
print(multiply.args)
print(multiply.return_direct)

calculator
useful for when you need to answer questions about math
{'a': {'title': 'A', 'description': 'first number', 'type': 'integer'}, 'b': {'title': 'B', 'description': 'second number', 'type': 'integer'}}
True


In [20]:
length = CustomLengthTool()
print(length.name)
print(length.description)
print(length.args)
print(length.return_direct)

length
Returns the length of a word.
{'query': {'title': 'Query', 'description': 'should be a string with one or many words', 'type': 'string'}}
True


In [22]:
length.invoke("today is a beautiful day to learn")

'33'

In [24]:
search.invoke("today is a beautiful day to learn")

'LangChain'

In [26]:
multiply.invoke({"a":4,"b":5})

20

In [23]:
tools = [search,multiply,length]

# Experiments

In [23]:
class ExtractFields(BaseModel):
    name: str
    age: int


chat = ChatFireworks(
    model="accounts/fireworks/models/firefunction-v2",
).bind_tools([ExtractFields])

result = chat.invoke("I am a 27 year old named Erick")

pprint(result.additional_kwargs["tool_calls"][0])

{'function': {'arguments': '{"name": "Erick", "age": 27}',
              'name': 'ExtractFields'},
 'id': 'call_cYfcB3JTTirtl6vXiRBumbFA',
 'index': 0,
 'type': 'function'}


In [25]:
chat = ChatFireworks(
    model="accounts/fireworks/models/firefunction-v2",
)

In [30]:
llm = ChatOpenAI(
    base_url="https://api.fireworks.ai/inference/v1",
    api_key="YOUR_FW_API_KEY",
    model="accounts/fireworks/models/firefunction-v1",
    temperature=0.0,
    max_tokens=256,
)

In [54]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [56]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [57]:
tool.name

'wikipedia'

In [58]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [59]:
tool.args

{'query': {'title': 'Query',
  'description': 'query to look up on wikipedia',
  'type': 'string'}}

In [60]:
tool.return_direct

False

In [61]:
tool.run({"query": "langchain"})

'Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications '

In [62]:
tool.run("langchain")

'Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications '

In [63]:
from langchain_core.pydantic_v1 import BaseModel, Field


class WikiInputs(BaseModel):
    """Inputs to the wikipedia tool."""

    query: str = Field(
        description="query to look up in Wikipedia, should be 3 or less words"
    )

In [64]:
tool = WikipediaQueryRun(
    name="wiki-tool",
    description="look up things in wikipedia",
    args_schema=WikiInputs,
    api_wrapper=api_wrapper,
    return_direct=True,
)

In [65]:
tool.name

'wiki-tool'

In [66]:
tool.description

'look up things in wikipedia'